In [35]:
def get_overall_reviews(driver, key_metrics, df):
    while True:
        
        driver.execute_script("window.scrollTo(1300, 1850);")
        try:
            elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located(
                    (By.CSS_SELECTOR, "div[class = 'v2__EIReviewDetailsV2__fullWidth v2__EIReviewDetailsV2__clickable']")))
        except:
            pass
        else:
            for i in range(0, len(elements)):
                    try:
                        elements[i].click()
                    except NoSuchElementException:
                        print("Element is not clickable")
                    else:
                        elements[i].click()
        finally:
            page = driver.page_source
            soup = BeautifulSoup(page)
            review_lst = soup.find_all('li', {'class':'noBorder empReview cf pb-0 mb-0'})
            for i in range(0, len(review_lst)):

                reviews = review_lst[i]
                title = reviews.find('h2', {"class":'mb-xxsm mt-0 css-93svrw el6ke055'}).get_text()
                status = reviews.find('span', {"class":'pt-xsm pt-md-0 css-1qxtz39 eg4psks0'}).get_text()
                date_position_txt = reviews.find('span', {'class':'middle common__EiReviewDetailsStyle__newGrey'}).get_text()
                pros = reviews.find('span', {'data-test':"pros"}).get_text()
                cons = reviews.find('span', {'data-test':"cons"}).get_text()
                try:
                    advice_to_mgmt = reviews.find('span', {'data-test':"advice-management"}).get_text()
                except:
                    advice_to_mgmt = None
                helpfulness = reviews.find('div', {'class':"common__EiReviewDetailsStyle__socialHelpfulcontainer pt-std"}).get_text()
                rating = reviews.find('span', {'class':'ratingNumber mr-xsm'}).get_text()

                df.loc[len(df)] = [title, status, date_position_txt, pros, cons, advice_to_mgmt, helpfulness, rating]

            # driver.maximize_window()
            # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # go to the next page
            try:
                next_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]/span')))

            except NoSuchElementException:
                print('no next button')
                break

            else:
                next_button.click()
                time.sleep(5)
                # WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located(
                #     (By.CSS_SELECTOR, "div[class = 'v2__EIReviewDetailsV2__fullWidth v2__EIReviewDetailsV2__clickable']")))
                                       
    return df

In [4]:
def position_type(x):

    position_type_lst = ['engineer', 'marketing', 'finance', 'sale', 'administrative', 'operation'
                         'consultant', 'analyst', 'human resource', 'product', 'director',
                         'account executive', 'data', 'user experience', 'recruiter' ,'dev', 'project']
    try: 
        x = x.lower()
    except:
        return None
    else:
        for ele in position_type_lst:
            if ele in x:
                return ele
            elif ele != position_type_lst:
                continue
            else:
                return None        


In [9]:
def get_benefit_reviews(key_metrics, df):
    while True:
        page = driver.page_source
        soup = BeautifulSoup(page)
        review_lst = soup.find_all('div', {'class':'pt-sm css-1ecrgor'})
        for i in range(0, len(review_lst)):

            reviews = review_lst[i]
            date = reviews.find('span', {'class':'css-1uyte9r', 'data-test':'review-date'}).get_text()
            text = reviews.find('p', {'class':'mt-std mb-std mt-sm-xsm'}).get_text()
            rating = reviews.find('strong', {'class':'mr-xxsm css-b63kyi'}).get_text()
            position = reviews.find('div', {'class':'mt-xxsm mt-sm-0 ml-0 ml-sm-xsm css-1uyte9r'}).get_text()
            df.loc[len(df)] = [date, text, rating, position]

        driver.maximize_window()
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # go to the next page
        try:
            next_button = driver.find_element(By.XPATH, '//*[@id="NodeReplace"]/div/div[3]/div[2]/div/div[1]/button[2]')
        
        except NoSuchElementException:
            break
        
        else:
            next_button.click()

    return df

In [ ]:
def status(x):
    if 'Current' in x:
        return 'Current'
    elif 'Former' in x:
        return 'Former'
    else:
        return None

In [ ]:
def position(x):
    if 'Current Employee' in x or 'Former Employee' in x:
        return 'unspecified'
    else:
        return x